# Data processing for director-firm samples
In this notebook I am creating a file that includes the control variables for each company and each year and I am matching the ISINs to those firms. In addition, I am identifying all relevant directors that sit on the board between 2011 and 2015 of all relevant companies (using the Refinitiv Eikon director data). Moreover, I run some sanity checks to see whether the director data is complete. Comparing the number of directors taken from the Refinitiv data to the board size data (also from Refinitiv), I can see that there are many inconsistencies (for details, scroll further down). However, it is not possible to manually confirm who is sitting on these boards, so I will accept the Refinitiv Eikon director data as is and assume that this represents the correct and true board composition per company and year. I will write the preprocessed director data and the committee data (also from Refinitiv Eikon) to csv files so that they can be used in the `biography_matching` notebook and the `csr_committees` notbook.

In [ ]:
# connecting to Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import os
import xlrd
import tqdm.notebook as tqdm
from functools import reduce


In [ ]:
# change settings to display full width of column content
pd.set_option('display.max_colwidth', None)
# set number of max rows
pd.set_option('display.max_rows', 13000)


In [ ]:
# Disable pandas warning about chained assignments
pd.options.mode.chained_assignment = None  # default='warn'


## Read in data

In [ ]:
# read in the founding year csv file which contains company tickers, names, and ISINs
founding_year_df = pd.read_csv('/content/drive/My Drive/director-csr/control_variables/founding_year.csv')
founding_year_df['Instrument'] = founding_year_df['Instrument'].apply(lambda x: x.split('.')[0])
founding_year_df.rename(columns={'ISIN Code': 'ISIN',
                                 'Instrument': 'ticker',
                                 '2016': 'part_of_2016',
                                 '2015': 'part_of_2015',
                                 '2014': 'part_of_2014',
                                 '2013': 'part_of_2013',
                                 '2012': 'part_of_2012',
                                 '2011': 'part_of_2011',
                                 '2010': 'part_of_2010'}, inplace=True)
founding_year_df.head()


,ticker,Organization Founded Year,RIC,Company Name,ISIN,part_of_2016,part_of_2015,part_of_2014,part_of_2013,part_of_2012,part_of_2011,part_of_2010
0,MMM,1929,MMM.N,3M Co,US88579Y1010,2016.0,2015.0,2014.0,2013.0,2012.0,2011.0,2010.0
1,ABT,1900,ABT.N,Abbott Laboratories,US0028241000,2016.0,2015.0,2014.0,2013.0,2012.0,2011.0,2010.0
2,ABBV,2012,ABBV.N,Abbvie Inc,US00287Y1091,2016.0,2015.0,2014.0,2013.0,NaN,NaN,NaN
3,ACN,2009,ACN.N,Accenture PLC,IE00B4BNMY34,2016.0,2015.0,2014.0,2013.0,2012.0,2011.0,NaN
4,ATVI,2000,ATVI.OQ,Activision Blizzard Inc,US00507V1098,2016.0,2015.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
def excel_read_in(file, list_of_dfs):
    '''
    Function to read in excel file and get the ticker from the file name
    and the company name from the first line of the excel file, then
    write the content to a pd.DataFrame
    '''
    # get the ticker from the file name
    ticker = os.path.basename(file).split('_')[0]
    # open the excel file and get the company name from the first row
    excel_sheet = xlrd.open_workbook(file)
    prim_sheet = excel_sheet.sheet_by_index(0)
    first_line = str(prim_sheet.row_slice(0)[0])[6:]
    company_name = first_line.split('|')[0].strip()
    # skipping the first 4 rows above the header
    df_file = pd.read_excel(file, skiprows=4)
    # add the company name and the ticker to the dataframe
    df_file['comp_name'] = company_name
    df_file['ticker'] = ticker
    list_of_dfs.append(df_file)

    return list_of_dfs
        

In [ ]:
# reading in the Excel files containing the directors per company
all_files = glob('/content/drive/My Drive/director-csr/directors_firms/*.xlsx')

all_directors_dfs = []
all_committees_dfs = []

# iterate through all file names
for file in tqdm.tqdm(all_files):

    # if the file name contains _dir
    if '_dir' in file:
        excel_read_in(file, all_directors_dfs)
    
    # if the file name contains _com
    elif '_com' in file:
        excel_read_in(file, all_committees_dfs)
        

## Preprocessing data 

In [ ]:
# list of tickers contained in the all_directors_dfs
all_tickers = []
for df in all_directors_dfs:
    all_tickers.append(df['ticker'].iloc[0].lower())

# list of all tickers contained in the founding_year_df
all_tickers_founding = list(founding_year_df['ticker'].str.lower())

# items contained in all_tickers list but not in the founding_year_df
tickers_no_founding = [x for x in all_tickers if x not in all_tickers_founding]
tickers_no_founding


['0ieo', 'bpyu', 'chkaq', 'cta', 'dofsq', 'ftc', 'ftrcq']

I need to manually check what company and ISIN these tickers could belong to.

In [ ]:
# check '0ieo' --> according to review, the old ticker was etfc
print('etfc' in all_tickers_founding)

# check 'bpyu' --> according to review, the old ticker was ggp
print('ggp' in all_tickers_founding)

# check 'chkaq' --> according to review, the old ticker was chk
print('chk' in all_tickers_founding)

# check 'cta' --> according to review, the old ticker was dd
print('dd' in all_tickers_founding)

# check 'dofsq' --> according to review, the old ticker was do
print('do' in all_tickers_founding)

# check 'ftc' --> there is no company that matches the founding year data

# check 'ftrcq' --> according to review, the old ticker was ftr
print('ftr' in all_tickers_founding)


True
True
True
True
True
True


In [ ]:
# get the list index for 'ftc'
for i, df in enumerate(all_directors_dfs):
    if df['ticker'].iloc[0].lower() == 'ftc':
        print(i)
        del all_directors_dfs[i]


201


In [ ]:
# there are two additional tickers which need to be removed from the drop_index for the founding_year_df
addtl_comps = ['aaba', 'adt']


In [ ]:
# replace all new tickers with old ones to match the founding_year_df tickers
# so that later on I can match ISINs to them
all_tickers_dir = []
all_tickers_com = []
# I have added the addtl_comps tickers to ensure that I get their list index and
# can remove them from the drop_index list
new_tickers = {'0ieo': 'etfc', 'bpyu': 'ggp', 'chkaq': 'chk', 'cta': 'dd',
               'dofsq': 'do', 'ftrcq': 'ftr', 'aaba': 'aaba', 'adt': 'adt'}
full_tickers_founding_year = []

for ticker in new_tickers.keys():
    # replace tickers for all director dfs
    for i, df in enumerate(all_directors_dfs):
        if df['ticker'].iloc[0].lower() == ticker:
            new_ticker = new_tickers[ticker]
            df['ticker'] = new_ticker
            all_tickers_dir.append(df['ticker'].iloc[0].lower())
            # get the RIC from the founding_year_df so that these can later be removed
            # from the drop_index list which contains tickers to be dropped
            full_tickers_founding_year.append(founding_year_df[founding_year_df['ticker'].str.lower() == new_ticker].index)

    # replace tickers for all committees dfs
    for df in all_committees_dfs:
        if df['ticker'].iloc[0].lower() == ticker:
            df['ticker'] = new_tickers[ticker]
            all_tickers_com.append(df['ticker'].iloc[0].lower())

# check whether there are any unknown tickers left
tickers_no_founding_dir = [x for x in all_tickers_dir if x not in all_tickers_founding]
tickers_no_founding_com = [x for x in all_tickers_com if x not in all_tickers_founding]
print(tickers_no_founding_dir)
print(tickers_no_founding_com)


[]
[]


In [ ]:
# drop all companies that have been delisted because I won't have director data for them
drop_index = list(founding_year_df[founding_year_df['RIC'].str.contains('\^')].index)
# remove items from that drop list which actually have data based on 
for element in full_tickers_founding_year:
    drop_index.remove(element)

# drop all remaining companies
founding_year_df.drop(index=drop_index, inplace=True)

# convert the content all relevant columns to lower case
for column in founding_year_df.columns:
    try:
        founding_year_df[column] = founding_year_df[column].str.lower()
    except:
        print('Column does not contain string values', column)

print(founding_year_df.shape)
founding_year_df.head()


Column does not contain string values Organization Founded Year
Column does not contain string values part_of_2016
Column does not contain string values part_of_2015
Column does not contain string values part_of_2014
Column does not contain string values part_of_2013
Column does not contain string values part_of_2012
Column does not contain string values part_of_2011
Column does not contain string values part_of_2010
(496, 12)


,ticker,Organization Founded Year,RIC,Company Name,ISIN,part_of_2016,part_of_2015,part_of_2014,part_of_2013,part_of_2012,part_of_2011,part_of_2010
0,mmm,1929,mmm.n,3m co,us88579y1010,2016.0,2015.0,2014.0,2013.0,2012.0,2011.0,2010.0
1,abt,1900,abt.n,abbott laboratories,us0028241000,2016.0,2015.0,2014.0,2013.0,2012.0,2011.0,2010.0
2,abbv,2012,abbv.n,abbvie inc,us00287y1091,2016.0,2015.0,2014.0,2013.0,NaN,NaN,NaN
3,acn,2009,acn.n,accenture plc,ie00b4bnmy34,2016.0,2015.0,2014.0,2013.0,2012.0,2011.0,NaN
4,atvi,2000,atvi.oq,activision blizzard inc,us00507v1098,2016.0,2015.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
# define the list of observed years
year_list = ['2011', '2012', '2013', '2014', '2015']


In [ ]:
# read in the file that contains board size
control_vars_df = pd.read_excel('/content/drive/My Drive/director-csr/control_variables/control_vars.xlsx')
# create separate df with only ISINs
control_vars_isin = control_vars_df[['ISIN Code']]
control_vars_isin['ISIN Code'] = control_vars_isin['ISIN Code'].str.lower()
control_vars_df.drop(columns=['ISIN Code'], inplace=True)
# drop rows that have all values missing
control_vars_df.dropna(how='all', inplace=True)
# convert the years from float to int
control_vars_df['Name'].astype(int)


0    2010
1    2011
2    2012
3    2013
4    2014
5    2015
6    2016
Name: Name, dtype: int64

In [ ]:
# check how many control variables are included per company
upper = '3m'.upper()
num_col_company_df = control_vars_df.filter(regex=upper)
print(num_col_company_df.shape)
# the first column containing the years needs to be subtracted
print((control_vars_df.shape[1]-1)/18)


(7, 18)
630.0


In [ ]:
# how many ISINs are included
print(control_vars_isin.shape)


(630, 1)


In [ ]:
# create a list of all ISINs that will be inserted as a row in the control_vars_df
all_isins_list = [item for isin in control_vars_isin['ISIN Code'] for item in [isin]*18]

# turn the Name column which contains the years into the index
control_vars_df['Name'] = control_vars_df['Name'].astype(int)
control_vars_df.set_index('Name', inplace=True)
#control_vars_df.index.name = 'year'

# add all ISINs to the table
control_vars_df.loc[0] = all_isins_list
control_vars_df


,3M - Value - Board Structure/Independent Board Members,3M - Value - Board Structure/Strictly Independent Board Members,3M - Board Size,3M - Value - Board Structure/CEO-Chairman Separation,3M - Value - Board Structure/Specific Skills,3M - RETURN ON ASSETS,3M - TOTAL DEBT % COMMON EQUITY,3M - TOTAL DEBT % TOTAL CAPITAL/STD,3M - TOTAL DEBT % TOTAL ASSETS,3M - NET SALES OR REVENUES,3M - NET SALES/REVENUES -1YR ANN GR,3M - NET SALES/REVES-3YR ANN GROWTH,3M - Compensation Policy Elements/Policy ESG Related Compensation,3M - Value - Board Structure/Board Diversity,3M - CASH FLOW/SALES,3M - CASH & EQUIVALENTS % TOTAL CUR,3M - Value - Board Structure/Experienced Board,3M - PRICE VOLATILITY,ABBOTT LABORATORIES - Value - Board Structure/Independent Board Members,ABBOTT LABORATORIES - Value - Board Structure/Strictly Independent Board Members,ABBOTT LABORATORIES - Board Size,ABBOTT LABORATORIES - Value - Board Structure/CEO-Chairman Separation,ABBOTT LABORATORIES - Value - Board Structure/Specific Skills,ABBOTT LABORATORIES - RETURN ON ASSETS,ABBOTT LABORATORIES - TOTAL DEBT % COMMON EQUITY,ABBOTT LABORATORIES - TOTAL DEBT % TOTAL CAPITAL/STD,ABBOTT LABORATORIES - TOTAL DEBT % TOTAL ASSETS,ABBOTT LABORATORIES - NET SALES OR REVENUES,ABBOTT LABORATORIES - NET SALES/REVENUES -1YR ANN GR,ABBOTT LABORATORIES - NET SALES/REVES-3YR ANN GROWTH,ABBOTT LABORATORIES - Compensation Policy Elements/Policy ESG Related Compensation,ABBOTT LABORATORIES - Value - Board Structure/Board Diversity,ABBOTT LABORATORIES - CASH FLOW/SALES,ABBOTT LABORATORIES - CASH & EQUIVALENTS % TOTAL CUR,ABBOTT LABORATORIES - Value - Board Structure/Experienced Board,ABBOTT LABORATORIES - PRICE VOLATILITY,ABBVIE - Value - Board Structure/Independent Board Members,ABBVIE - Value - Board Structure/Strictly Independent Board Members,ABBVIE - Board Size,ABBVIE - Value - Board Structure/CEO-Chairman Separation,...,RS LEGACY CORP - CASH FLOW/SALES,RS LEGACY CORP - CASH & EQUIVALENTS % TOTAL CUR,RS LEGACY CORP - Value - Board Structure/Experienced Board,RS LEGACY CORP - PRICE VOLATILITY,SUNEDISON - Value - Board Structure/Independent Board Members,SUNEDISON - Value - Board Structure/Strictly Independent Board Members,SUNEDISON - Board Size,SUNEDISON - Value - Board Structure/CEO-Chairman Separation,SUNEDISON - Value - Board Structure/Specific Skills,SUNEDISON - RETURN ON ASSETS,SUNEDISON - TOTAL DEBT % COMMON EQUITY,SUNEDISON - TOTAL DEBT % TOTAL CAPITAL/STD,SUNEDISON - TOTAL DEBT % TOTAL ASSETS,SUNEDISON - NET SALES OR REVENUES,SUNEDISON - NET SALES/REVENUES -1YR ANN GR,SUNEDISON - NET SALES/REVES-3YR ANN GROWTH,SUNEDISON - Compensation Policy Elements/Policy ESG Related Compensation,SUNEDISON - Value - Board Structure/Board Diversity,SUNEDISON - CASH FLOW/SALES,SUNEDISON - CASH & EQUIVALENTS % TOTAL CUR,SUNEDISON - Value - Board Structure/Experienced Board,SUNEDISON - PRICE VOLATILITY,TELLABS - Value - Board Structure/Independent Board Members,TELLABS - Value - Board Structure/Strictly Independent Board Members,TELLABS - Board Size,TELLABS - Value - Board Structure/CEO-Chairman Separation,TELLABS - Value - Board Structure/Specific Skills,TELLABS - RETURN ON ASSETS,TELLABS - TOTAL DEBT % COMMON EQUITY,TELLABS - TOTAL DEBT % TOTAL CAPITAL/STD,TELLABS - TOTAL DEBT % TOTAL ASSETS,TELLABS - NET SALES OR REVENUES,TELLABS - NET SALES/REVENUES -1YR ANN GR,TELLABS - NET SALES/REVES-3YR ANN GROWTH,TELLABS - Compensation Policy Elements/Policy ESG Related Compensation,TELLABS - Value - Board Structure/Board Diversity,TELLABS - CASH FLOW/SALES,TELLABS - CASH & EQUIVALENTS % TOTAL CUR,TELLABS - Value - Board Structure/Experienced Board,TELLABS - PRICE VOLATILITY
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010,90,60,10,Y,70,14.68,34.81,25.39,18.48,2.6662e+07,15.31,2.91,Y,20,18.9,36.97,8.38,18.64,87.5,NaN,15,Y,18.75,9.32,81.6,44.84,31.06,3.51667e+07,14.31,10.71,N,13.33,22.97,32.82,10.63,14.45,NaN,NaN,NaN,NaN,...,7.69,32.12,12.71,32.17,90,90,10,N,100,1

In [ ]:
# check whether there are any duplicate tickers in founding_year_df
founding_year_df[founding_year_df['ticker'].duplicated()]


,ticker,Organization Founded Year,RIC,Company Name,ISIN,part_of_2016,part_of_2015,part_of_2014,part_of_2013,part_of_2012,part_of_2011,part_of_2010


In [ ]:
# get a list of all the available variables
all_vars_headers = list(control_vars_df.columns[:18])
all_vars = [x.split('-')[-1].strip() for x in all_vars_headers]
print(len(all_vars))
all_vars


18


['Board Structure/Independent Board Members',
 'Board Structure/Strictly Independent Board Members',
 'Board Size',
 'Chairman Separation',
 'Board Structure/Specific Skills',
 'RETURN ON ASSETS',
 'TOTAL DEBT % COMMON EQUITY',
 'TOTAL DEBT % TOTAL CAPITAL/STD',
 'TOTAL DEBT % TOTAL ASSETS',
 'NET SALES OR REVENUES',
 '1YR ANN GR',
 '3YR ANN GROWTH',
 'Compensation Policy Elements/Policy ESG Related Compensation',
 'Board Structure/Board Diversity',
 'CASH FLOW/SALES',
 'CASH & EQUIVALENTS % TOTAL CUR',
 'Board Structure/Experienced Board',
 'PRICE VOLATILITY']

In [ ]:
def yearly_controls(str_control, year):
    '''
    Function to create separate dataframes per year for all companies
    and control variables
    '''

    # get all columns that contain particular control variable
    one_control_df = control_vars_df.filter(regex=str_control)
    # print the number of columns indicating the number of companies with that data
    
    # transpose the dataframe
    one_control_T = one_control_df.T
    one_control_T.reset_index(inplace=True)
    one_year_df = one_control_T[['index', year, 0]].copy()

    # change the column names
    one_year_df.rename(columns={'index': 'company', year: str_control, 0: 'ISIN'}, inplace=True)

    # drop the column of company name because the same isin may have different company names
    one_year_df.drop(columns='company', inplace=True)

    return one_year_df


In [ ]:
# create dataframes for each year and control variable
all_years_dict = {}
# create a variable that captures the overall size of all combined dataframes
# this will serve as a sanity check to confirm that the merging has correctly worked
all_dfs_length = 0

# iterate through all years that will be observed
for year in year_list:
    # define an empty list to save all generated dfs in
    list_dfs_control_vars = []
    # iterate through all observed control variables
    for variable in all_vars:
        # generate the dfs containing the variable for a specific year
        df = yearly_controls(variable, int(year))
        # add this df to a list
        list_dfs_control_vars.append(df)

    # merge the dataframes for the same year
    all_vars_for_year = reduce(lambda left, right: pd.merge(left, right, on=['ISIN'], # merge just on isin
                                            how='outer'), list_dfs_control_vars)
    
    # add the founding_year_df to the all_years_dict entries
    all_vars_founding_year = pd.merge(founding_year_df, all_vars_for_year, on=['ISIN'], how='outer')

    # add a column that contains the year these samples related to
    all_vars_founding_year['year'] = year

    # add the merged dataframes to a list
    all_years_dict[year] = all_vars_founding_year

    # add the shape of the dataframe to the overall length variable
    all_dfs_length += all_vars_founding_year.shape[0]


In [ ]:
# now merge all the dataframes containing control variables for different years
# into one overall dataframe
for key, val in all_years_dict.items():
    if key == '2011':
        all_control_vars_df = val.copy()
    else:
        all_control_vars_df = all_control_vars_df.append(val)


In [ ]:
# check the shapes of the dataframes
print('Overall dataframe:', all_control_vars_df.shape[0])
print('Individual combined dataframes:', all_dfs_length)


Overall dataframe: 3145
Individual combined dataframes: 3145


In [ ]:
# write control variable data to csv files
all_control_vars_df.to_csv('/content/drive/My Drive/director-csr/control_variables/all_control_vars.csv')


Independent directors will be considered for their entire tenure at the organizations, not just for in position, because by definition, independent directors cannot be former employees, therefore, they could not have held executive positions prior to becoming independent directors. The reason I will consider this is because in the case of fortune brands home & security inc there is a lead independent director which held a position prior to that. However, the prior position is not named. Yet, checking perid.org shows that he was previously an independent (not lead) director at this company.

In [ ]:
def preprocess(df, committee=False):
    '''
    Function to preprocess dataframes
    Args: df = pd.DataFrame
          committee = boolean
    Returns: df = pd.DataFrame
    '''
    # replace all -- with NaN
    df = df.replace('--', np.nan)

    # convert all entries to lower case
    for column in df.columns:
        # applying lower() to the age column converts all ages to NaN, therefore,
        # this column will be skipped when converting content to lower case
        if column == 'Alter':
            pass
        else:
            df[column] = df[column].str.lower()     

    try:   
        # define the new columns depending on whether this is director or committee data
        if committee:
            new_cols_current = ['name', 'committee', 'age', 'current_position', 'dir_exec', 'in_position']
            new_cols_former = ['name', 'committee', 'age', 'last_position', 'director_start', 'director_end', 'executive_start', 'executive_end']
            current_df, former_df = np.split(df, df.index[df[df.columns[:-2]].isnull().all(1)])

            # drop Gesamtvergütung column and Unnamed column
            if former_df.iloc[2]['Name'] != 'name':
                current_df = current_df.drop(current_df.columns[[-3]], axis=1)
            else: 
                current_df = current_df.drop(current_df.columns[[-4, -3]], axis=1)

        else:
            new_cols_current = ['name', 'age', 'current_position', 'dir_exec', 'in_position']
            new_cols_former = ['name', 'age', 'last_position', 'director_start', 'director_end', 'executive_start', 'executive_end']
            # split the dataframe on the line that contains NaN in the Name column
            current_df, former_df = np.split(df, df[df.isnull().Name].index)
            
            # some tickers have Gesamtvergütung with pound sign, not dollar sign
            if 'Gesamtvergütung ($)' in current_df.columns:
                current_df = current_df.drop(columns=['Gesamtvergütung ($)', 'Unnamed: 6'])
            else:
                current_df = current_df.drop(current_df.columns[[-4, -3]], axis=1)

        # define new column names
        current_df = current_df.rename(columns=dict(zip(current_df.columns[:-2], new_cols_current)))

        # drop empty rows and rename columns
        if former_df.iloc[2]['Name'] == 'name':
            former_df_index = former_df.reset_index(drop=True).loc[2:, :]
            # define new column names
            former_df_index = former_df_index.rename(columns=dict(zip(former_df_index.columns[:-2], new_cols_former)))
            former_df = former_df_index.iloc[1:]
        else: 
            try:
                print ('This company has a problematic layout of the former director table:',
                former_df['ticker'][0])       
            except:
                print('The former_df dataframe is empty')
                former_df = pd.DataFrame(data=None)

        # drop all directors that did not hold director positions (because Reuters incorrectly
        # translated the fields, I need to look at the executive start and end fields)
        if committee:
            pass
        else:
            former_df = former_df.dropna(subset=['executive_start', 'executive_end'], how='all')

        # if the former_df is empty
        if former_df.empty:
            pass
        else:
            # convert dates to just years
            for col_name in ['director_start', 'director_end', 'executive_start', 'executive_end']:
                former_df[col_name] = former_df[col_name].apply(lambda x: 0 if isinstance(x, float)
                                                                        else x[-4:])
            # flag the directors without start date but only an end date after 2009
            former_df['missing_start_date'] = former_df.apply(lambda x: 1 if int(x['executive_start']) == 0 and int(x['executive_end']) > 2009 
                                                                        else 0, axis=1)
            
            # add columns for each year
            for year in year_list:
                # I will only consider the executive positions for the year fields
                former_df[year] = former_df.apply(lambda x: 1 if int(x['executive_start']) <= int(year) and int(x['executive_start']) != 0 
                                                                        and int(x['executive_end']) >= int(year) else 0, axis=1)

        # isolate number of years from dir_exec and in_position columns
        current_df['dir_exec'] = current_df['dir_exec'].apply(lambda x: 0 if isinstance(x, float)
                                                                        else (2021 - int(x.split(' ')[1]) if len(x.split(' ')) == 3
                                                                              else 2021 - int(x.split(' ')[0])))
        current_df['in_position'] = current_df['in_position'].apply(lambda x: 0 if isinstance(x, float)
                                                                              else (2021 - int(x.split(' ')[1]) if len(x.split(' ')) == 3
                                                                                    else 2021 - int(x.split(' ')[0])))

        # drop people in director dataframes
        if committee:
            current_df_new = current_df.copy()
        else:
            # drop all people that held positions other than directors
            df_combined = current_df[current_df['current_position'].str.contains('director|chairman|independent|non-executive', regex=True)]
            # drop any current directors whose dir_exec column holds a value greater than 2016
            current_df_new = df_combined[df_combined['dir_exec'] < 2017]

        # don't do anything if there are no directors currently on the board who were appointed before 2017
        if current_df_new.empty:
            pass
        else:
            # add columns for each year
            for year in year_list:
                # I will only consider the current in_position field for the year columns
                current_df_new[year] = current_df_new.apply(lambda x: 1 if int(x['in_position']) <= int(year) and int(x['in_position']) != 0
                                                                        else 0, axis=1)
                # if there is an independent director with a prior role, I will also consider this prior role because they likely also were director
                current_df_new[year] = current_df_new.apply(lambda x: 1 if 'independent' in x['current_position'] and int(x['dir_exec']) <= int(year) and int(x['dir_exec']) != 0
                                                                    else x[year], axis=1)

        # compress committee membership into list per director
        if committee:
            # do the below iterations for both former and current dfs
            both_dfs = [current_df_new, former_df]
            for df in both_dfs:
                # don't do anything if the former_df dataframe is empty
                if df.empty:
                    pass
                else:
                    committees_start_end = list(df.index[df['committee'].isnull()])
                    # add the length of the committee column to the index list to ensure that
                    # the last director also gets his/her committees assigned
                    committees_start_end.append(df['committee'].index[-1]+1)
                    for i in range(len(committees_start_end)-1):
                        committee_memb = list(df['committee'].loc[committees_start_end[i]+1 : committees_start_end[i+1]-1])
                        df['committee'].loc[committees_start_end[i]] = committee_memb

                    # drop all rows that have NaNs in the name column or 'komitee-mitgliedschaft'
                    df.drop(df[(df['name'].isnull()) | (df['name'] == 'komitee-mitgliedschaft')].index, inplace=True)

        else:
            pass

        return current_df_new, former_df 

    except:
        print('There seems to be an issue with this dataframe:', df['ticker'][0])

        return df['ticker'][0], df['ticker'][0]


In [ ]:
def combine_isin(df_list, committee=False):
    '''
    Function to split all dfs, structure them and then add isin to them
    Args: df_list = df,
          committee = boolean
    Returns: all_director_dfs = df
    '''
    # iterate through all dfs and split and structure them
    all_current_dfs = []
    all_former_dfs = []
    all_director_dfs = []
    removal_index = None

    for i, df in enumerate(df_list):
        current_df, former_df = preprocess(df, committee=committee)
        all_current_dfs.append(current_df)
        all_former_dfs.append(former_df)

        try:
            df_combined = pd.concat([former_df, current_df])
        except:
            print('This list item is not a dataframe but only the ticker')

        # add a column with the ISIN to all combined dataframes
        issues_list = []
        try:
            ticker = list(df_combined['ticker'])[0]
            isin = founding_year_df[founding_year_df['ticker'] == ticker]['ISIN']
        except:
            print('This dataframe seems to be empty', df_combined, i)
            removal_index = i

        try:
            df_combined['isin'] = isin.values[0]
        except:
            issues_list.append(i)
            print('There are issues with this ticker:', ticker, i)

        all_director_dfs.append(df_combined)

    return all_director_dfs, removal_index, issues_list


In [ ]:
# apply structure all director and committee dfs and apply isins
all_combined_dfs, removal_index, issues_list = combine_isin(all_directors_dfs, committee=False)
all_committees_dfs, removal_index_comm, issues_list_comm = combine_isin(all_committees_dfs, committee=True)


This dataframe seems to be empty Empty DataFrame
Columns: [name, age, last_position, director_start, director_end, executive_start, executive_end, comp_name, ticker, current_position, dir_exec, in_position]
Index: [] 280
There seems to be an issue with this dataframe: etfc
This list item is not a dataframe but only the ticker
The former_df dataframe is empty
The former_df dataframe is empty
The former_df dataframe is empty
The former_df dataframe is empty
The former_df dataframe is empty


I will disregard the ETFC company for now because it was delisted.

In [ ]:
# drop the empty dataframe
del all_combined_dfs[removal_index]


Based on the fact that the biography data is from 2015, I will look at data from 2011 until 2016 in my analysis. The control and indepedent variables need to range from 2011 until 2015, while the dependent variable needs to range from 2012 until 2016.

I need to systematically compare board size with the number of directors filtered.

In [ ]:
def check_dir_size(df_list):
    '''
    Function to check whether the number of filtered directors is
    the same as the board size
    '''

    too_high = []
    too_low = []

    for company_index in range(len(df_list)):
        # if the current company is part of the issues_list generated earlier, skip it
        if company_index in issues_list:
            pass

        else:
            # get the company isin and name
            company_isin = df_list[company_index]['isin'].iloc[0]
            company_name = df_list[company_index]['comp_name'].iloc[0]

            # check for all years covered
            for year in year_list:

                # get the board size from the control variables df
                comp_filtered = list(all_years_dict[year][all_years_dict[year]['ISIN'] == company_isin]['Board Size'])
                comp_board_size = [x for x in comp_filtered if not pd.isna(x)]

                # get the number of directors
                num_dir_year = df_list[company_index][df_list[company_index][year] == 1].shape[0]
                
                # try to check whether the number of directors in a given year is the same
                # as the board size given in the control variables df
                try:
                    
                    if len(comp_board_size) > 0:
                        
                        if num_dir_year == comp_board_size[0]:
                            pass
                        else:                            
                            # add incorrect matches if they are more than 1 greater or smaller than board size
                            if num_dir_year > comp_board_size[0]:
                                # check whether there is a director that has the same start and end year
                                same_start_end = list(df_list[company_index][df_list[company_index][['executive_start','executive_end']].nunique(axis=1) == 1]['executive_start'])

                                if len(same_start_end) > 0 and year in same_start_end:
                                    print('Less than one year term:', year, company_name)

                                else:
                                    too_high.append(company_index)
                                    print('Too high:', company_name, year,
                                            'directors:', num_dir_year,
                                            'board size:', comp_board_size[0], 
                                            'company index:', company_index)
                                    
                            elif num_dir_year < comp_board_size[0]:
                                # check whether there are any directors with a missing start date
                                missing_start_date = df_list[company_index][df_list[company_index]['missing_start_date'] == 1][['name', 'last_position', 'executive_end']]
                                
                                if any(int(x) > 2010 for x in list(missing_start_date['executive_end'])):
                                #for index, row in missing_start_date.iterrows():
                                 #   if int(row['executive_end']) > 2010:
                                    print('Missing start date', year, company_name)

                                else:
                                    too_low.append(company_index)
                                    print('Too low:', company_name, year,
                                            'directors:', num_dir_year,
                                            'board size:', comp_board_size[0], 
                                            'company index:', company_index)

                    else:
                        pass

                except:
                    print('There seems to be an issue with the board size column in the control variables df:',
                        company_name)
                    
    return too_high, too_low




In [ ]:
# check the board size
too_high, too_low = check_dir_size(all_combined_dfs)


Missing start date 2011 itt inc
Missing start date 2012 itt inc
Missing start date 2013 itt inc
Missing start date 2014 itt inc
Missing start date 2015 itt inc
Too low: meredith corp 2011 directors: 9 board size: 10.0 company index: 1
Too high: meredith corp 2014 directors: 9 board size: 8.0 company index: 1
Too high: willis towers watson plc 2012 directors: 12 board size: 11.0 company index: 2
Too high: willis towers watson plc 2013 directors: 15 board size: 12.0 company index: 2
Too high: willis towers watson plc 2015 directors: 12 board size: 6.0 company index: 2
Too low: udr inc 2011 directors: 8 board size: 9.0 company index: 3
Too low: udr inc 2013 directors: 7 board size: 8.0 company index: 3
Too low: ulta beauty inc 2011 directors: 4 board size: 5.0 company index: 4
Too low: ulta beauty inc 2012 directors: 6 board size: 7.0 company index: 4
Too low: ulta beauty inc 2014 directors: 9 board size: 10.0 company index: 4
Too low: ulta beauty inc 2015 directors: 10 board size: 11.0 c

I think that Reuters incorrectly labeled the fields relating to the time as a director and as an executive. It seems like they mixed them up. Therefore, I will only focus on the executive_start and executive_end field.

It also seems like pretty much every company has a different reported board size compared to its listed directors.

When I look at the share of incorrect matches between board size and director number, only roughly a third of all company-year combinations include correct matches. If I consider a variance of 1, then about 80% are correct. I think it is reasonable to consider these variances of 1 as acceptable because some directors may only be on the board a couple of months and the board size - which relates to the size at the end of the fiscal year - does not capture them then.

What I could also consider is predicting all biographies, and then really only focus on the ones that include social or environmental matters. If all of those are correct and the board size mostly matches the number of directors, then I should be fine as well.

In [ ]:
# there are 489 companies and 5 years minus a couple of companies with some issues
# but roughly this amounts to a total of 2,445 potential entries in the too_high list
# the below number shows the portions of years with higher numbers of directors than board size
len(too_high)/2445


0.29243353783231085

In [ ]:
# the same can be done for the too_low list
len(too_low)/2445


0.2519427402862986

In [ ]:
# check whether there are directors that have the same start and end date
# indicating that they were not counted for the fiscal year end board size metric
count = 0
for i, company in enumerate(all_combined_dfs):
    df_temp = company[company[['executive_start','executive_end']].nunique(axis=1) == 1]
    if not df_temp.empty:
        count += 1
print(count)

# I implemented this in the function check_dir_size and this has an effect of around 1 %age point less issues


92


In [ ]:
# these cases I need to check manually to add missing start dates for directors
count = 0
for company in all_combined_dfs:
    missing_start_date = company[company['missing_start_date'] == 1][['name', 'last_position', 'executive_end', 'comp_name']]
    #for index, row in missing_start_date.iterrows():
    if any(int(x) > 2010 for x in list(missing_start_date['executive_end'])):
        print(missing_start_date)
        count += 1
print('This many companies had directors with missing start dates:', count)

# I implemented this in the function check_dir_size and this has an effect of around 5 %age point less issues


                    name  ... comp_name
12       ms. karen larue  ...   itt inc
14  mr. g. peter d'aloia  ...   itt inc

[2 rows x 4 columns]
                 name  ...                     comp_name
19  ms. ellen alemany  ...  citizens financial group inc

[1 rows x 4 columns]
                          name  ...               comp_name
5  mr. jaime ellertson , ph.d.  ...  e*trade financial corp

[1 rows x 4 columns]
                 name last_position executive_end comp_name
5   mr. timothy whall      director          2019   adt inc
10   mr. brett watson      director          2018   adt inc
               name  ...    comp_name
11  mr. scott cisel  ...  ameren corp

[1 rows x 4 columns]
               name  ...                               comp_name
7  mr. devin murphy  ...  apartment investment and management co
9    mr. john rayis  ...  apartment investment and management co

[2 rows x 4 columns]
                      name         last_position executive_end     comp_name
5   mr. 

These 38 cases could be checked manually and then the start date could be added manually.


### Get all directors included in my dataset

In [ ]:
# create dataframe that only contains director names, company names, company tickers, years, and ISINs
for i, df in enumerate(all_combined_dfs):
    try:
        df = df[['name', 'isin', 'comp_name', 'ticker', '2011', '2012', '2013', '2014', '2014']]

    except:
        print(i, df['ticker'][0])


In [ ]:
# concat all data frames in all_combined_dfs to prepare for writing to csv file
all_final_directors_dfs = pd.concat(all_combined_dfs, ignore_index=True)
all_final_directors_dfs


In [ ]:
# check how many unique directors are included
all_final_directors_dfs['name'].nunique()


9465

In [ ]:
# write data to csv files
all_final_directors_dfs.to_csv('/content/drive/My Drive/director-csr/all_directors.csv')


## CSR committee data


In [ ]:
# concat all data frames in all_committees_dfs to prepare for writing to csv file
all_final_committees_dfs = pd.concat(all_committees_dfs, ignore_index=True)
all_final_committees_dfs


In [ ]:
# write data to csv files
all_final_committees_dfs.to_csv('/content/drive/My Drive/director-csr/all_committees.csv')
